# Solve Business Problems with AI

## Objective
Develop a proof-of-concept application to intelligently process email order requests and customer inquiries for a fashion store. The system should accurately categorize emails as either product inquiries or order requests and generate appropriate responses using the product catalog information and current stock status.

You are encouraged to use AI assistants (like ChatGPT or Claude) and any IDE of your choice to develop your solution. Many modern IDEs (such as PyCharm, or Cursor) can work with Jupiter files directly.

## Task Description

### Inputs

Google Spreadsheet **[Document](https://docs.google.com/spreadsheets/d/14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U)** containing:

- **Products**: List of products with fields including product ID, name, category, stock amount, detailed description, and season.

- **Emails**: Sequential list of emails with fields such as email ID, subject, and body.

### Instructions

- Implement all requirements using advanced Large Language Models (LLMs) to handle complex tasks, process extensive data, and generate accurate outputs effectively.
- Use Retrieval-Augmented Generation (RAG) and vector store techniques where applicable to retrieve relevant information and generate responses.
- You are provided with a temporary OpenAI API key granting access to GPT-4o, which has a token quota. Use it wisely or use your own key if preferred.
- Address the requirements in the order listed. Review them in advance to develop a general implementation plan before starting.
- Your deliverables should include:
   - Code developed within this notebook.
   - A single spreadsheet containing results, organized across separate sheets.
   - Comments detailing your thought process.
- You may use additional libraries (e.g., langchain) to streamline the solution. Use libraries appropriately to align with best practices for AI and LLM tools.
- Use the most suitable AI techniques for each task. Note that solving tasks with traditional programming methods will not earn points, as this assessment evaluates your knowledge of LLM tools and best practices.

### Requirements

#### 1. Classify emails
    
Classify each email as either a _**"product inquiry"**_ or an _**"order request"**_. Ensure that the classification accurately reflects the intent of the email.

**Output**: Populate the **email-classification** sheet with columns: email ID, category.

#### 2. Process order requests
1.   Process orders
  - For each order request, verify product availability in stock.
  - If the order can be fulfilled, create a new order line with the status “created”.
  - If the order cannot be fulfilled due to insufficient stock, create a line with the status “out of stock” and include the requested quantity.
  - Update stock levels after processing each order.
  - Record each product request from the email.
  - **Output**: Populate the **order-status** sheet with columns: email ID, product ID, quantity, status (**_"created"_**, **_"out of stock"_**).

2.   Generate responses
  - Create response emails based on the order processing results:
      - If the order is fully processed, inform the customer and provide product details.
      - If the order cannot be fulfilled or is only partially fulfilled, explain the situation, specify the out-of-stock items, and suggest alternatives or options (e.g., waiting for restock).
  - Ensure the email tone is professional and production-ready.
  - **Output**: Populate the **order-response** sheet with columns: email ID, response.

#### 3. Handle product inquiry

Customers may ask general open questions.
  - Respond to product inquiries using relevant information from the product catalog.
  - Ensure your solution scales to handle a full catalog of over 100,000 products without exceeding token limits. Avoid including the entire catalog in the prompt.
  - **Output**: Populate the **inquiry-response** sheet with columns: email ID, response.

## Evaluation Criteria
- **Advanced AI Techniques**: The system should use Retrieval-Augmented Generation (RAG) and vector store techniques to retrieve relevant information from data sources and use it to respond to customer inquiries.
- **Tone Adaptation**: The AI should adapt its tone appropriately based on the context of the customer's inquiry. Responses should be informative and enhance the customer experience.
- **Code Completeness**: All functionalities outlined in the requirements must be fully implemented and operational as described.
- **Code Quality and Clarity**: The code should be well-organized, with clear logic and a structured approach. It should be easy to understand and maintain.
- **Presence of Expected Outputs**: All specified outputs must be correctly generated and saved in the appropriate sheets of the output spreadsheet. Ensure the format of each output matches the requirements—do not add extra columns or sheets.
- **Accuracy of Outputs**: The accuracy of the generated outputs is crucial and will significantly impact the evaluation of your submission.

We look forward to seeing your solution and your approach to solving real-world problems with AI technologies.

### Installing Required Library

In [ ]:
!pip install \
    openai==1.14.3 \
    faiss-cpu \
    gspread \
    gspread_dataframe \
    tiktoken \
    numpy==1.26.4 \
    anyio==3.7.1 \
    packaging==24.2.0 \
    httpx==0.28.1 \
    pandas \
    -q --upgrade --force-reinstall




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.7/792.7 kB 30.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.0.2 requires tiktoken<0.6.0,>=0.5.2, but you have tiktoken 0.9.0 which is incompatible.
langchain-core 0.1.23 requires langsmith<0.0.88,>=0.0.87, but you have langsmith 0.0.92 which is incompatible.
langchain-core 0.1.23 requires packaging<24.0,>=23.2, but you have packaging 24.2 which is incompatible.
google-colab 1.0.0 requires google-auth==2.38.0, but you have google-auth 2.39.0 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-c

In [ ]:
!pip install langchain==0.0.340 langchain-core==0.1.23 langsmith==0.0.92 --force-reinstall --no-deps

  Using cached langchain-0.0.340-py3-none-any.whl.metadata (16 kB)
  Using cached langchain_core-0.1.23-py3-none-any.whl.metadata (6.0 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
Using cached langchain-0.0.340-py3-none-any.whl (2.0 MB)
Using cached langchain_core-0.1.23-py3-none-any.whl (241 kB)
Using cached langsmith-0.0.92-py3-none-any.whl (56 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.92
    Uninstalling langsmith-0.0.92:
      Successfully uninstalled langsmith-0.0.92
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.23
    Uninstalling langchain-core-0.1.23:
      Successfully uninstalled langchain-core-0.1.23
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.340
    Uninstalling langchain-0.0.340:
      Successfully uninstalled langchain-0.0.340


### Google Authentication and Setup

In [ ]:
# Authenticate with Google and load required libraries
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe
import pandas as pd

# Authenticate
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Load spreadsheet from URL and fetch the 'products' and 'emails' tabs into Pandas DataFrames
input_sheet_url = 'https://docs.google.com/spreadsheets/d/14fKHsblfqZfWj3iAaM2oA51TlYfQlFT4WKo52fVaQ9U'
spreadsheet = gc.open_by_url(input_sheet_url)

# Read 'products' tab
products_ws = spreadsheet.worksheet('products')
catalog_df = pd.DataFrame(products_ws.get_all_records())

# Read 'emails' tab
emails_ws = spreadsheet.worksheet('emails')
emails_df = pd.DataFrame(emails_ws.get_all_records())

# Display first 3 rows of each DataFrame
display(catalog_df.head(3))
display(emails_df.head(3))

,product_id,name,category,description,stock,seasons,price
0,RSG8901,Retro Sunglasses,Accessories,Transport yourself back in time with our retro...,1,"Spring, Summer",26.99
1,SWL2345,Sleek Wallet,Accessories,Keep your essentials organized and secure with...,5,All seasons,30.00
2,VSC6789,Versatile Scarf,Accessories,Add a touch of versatility to your wardrobe wi...,6,"Spring, Fall",23.00


,email_id,subject,message
0,E001,Leather Wallets,"Hi there, I want to order all the remaining LT..."
1,E002,Buy Vibrant Tote with noise,"Good morning, I'm looking to buy the VBT2345 V..."
2,E003,Need your help,"Hello, I need a new bag to carry my laptop and..."


### Setting Up OpenAi Auth and Helper Function for Chat Completion

In [ ]:
# Setup OpenAI API with provided GPT-4o base URL

from openai import OpenAI
from google.colab import userdata



client = OpenAI(
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/',
    api_key='a0BIj000002bRQmMAM'
)
# Helper to simplify prompt and response interaction with GPT
def ask_gpt(prompt, system_message=None):
    messages = []
    if system_message:
        messages.append({"role": "system", "content": system_message})
    messages.append({"role": "user", "content": prompt})

    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )
    return completion.choices[0].message.content.strip()

### Vector Store for Product RAG

In [ ]:
# First file: Create and save the vector store
import os
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain_openai import OpenAIEmbeddings

# Path where the FAISS vector store will be saved
VECTOR_STORE_PATH = "faiss_store"

# Initialize OpenAIEmbeddings with your API key and base URL
embeddings = OpenAIEmbeddings(
    api_key='a0BIj000002bRQmMAM',
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/'
)

# Create a wrapper function to make embeddings callable
class EmbeddingWrapper:
    def __init__(self, embeddings):
        self.embeddings = embeddings

    def __call__(self, text):
        return self.embeddings.embed_query(text)

# Use the wrapper
embedding_function = EmbeddingWrapper(embeddings)

# Check if the FAISS vector store already exists
if os.path.exists(VECTOR_STORE_PATH):
    # Load the existing FAISS vector store - note the parameter name is 'embeddings'
    db = FAISS.load_local(VECTOR_STORE_PATH, embeddings=embedding_function)
else:
    # Convert product rows to LangChain Document objects for vectorization
    product_docs = [
        Document(
            page_content=f"Product: {row['name']}, Description: {row['description']}, Category: {row['category']}, Season: {row['seasons']}, Price: {row['price']}",
            metadata={"product_id": row['product_id'], "stock": row['stock']}
        ) for _, row in catalog_df.iterrows()
    ]

    # Create FAISS vector store from the product documents
    texts = [doc.page_content for doc in product_docs]
    metadatas = [doc.metadata for doc in product_docs]

    # Note: The parameter name for from_texts is 'embedding'
    db = FAISS.from_texts(texts, embedding=embedding_function, metadatas=metadatas)

    # Save the FAISS vector store locally for future use
    db.save_local(VECTOR_STORE_PATH)

# Task 1. Classify emails

In [ ]:
#Email Classification
email_classification = []
for _, row in emails_df.iterrows():
    prompt = f"Classify this email as 'product inquiry' or 'order request':\nSubject: {row['subject']}\nMessage: {row['message']}"
    category = ask_gpt(prompt)
    email_classification.append({"email ID": row['email_id'], "category": category.lower().strip()})

email_classification_df = pd.DataFrame(email_classification)

# Task 2. Process order requests

In [ ]:
# Order Processing
order_status = []
order_responses = []
# Create a mapping of product stock by ID
stock_map = {row['product_id']: int(row['stock']) for _, row in catalog_df.iterrows()}

for _, row in emails_df.iterrows():
    email_id, subject, message = row['email_id'], row['subject'], row['message']
    category_row = next((item for item in email_classification if item['email ID'] == email_id), None)
    if category_row and category_row['category'] == 'order request':
        # Extract products and quantities using GPT
        prompt = f"Extract ordered product names and quantities from this message:\n{message}\nReturn as: product_name: quantity"
        response = ask_gpt(prompt)

        order_lines = []
        for line in response.split("\n"):
            if ":" in line:
                try:
                    name, qty = line.split(":")
                    name, qty = name.strip().lower(), int(qty.strip())
                    # Match product name to catalog
                    matched_product = catalog_df[catalog_df['name'].str.lower() == name]
                    if not matched_product.empty:
                        product_id = matched_product.iloc[0]['product_id']
                        available = stock_map[product_id]
                        # Check stock availability
                        if available >= qty:
                            status = 'created'
                            stock_map[product_id] -= qty   # Reducing stock
                        else:
                            status = 'out of stock'
                        order_status.append({
                            'email ID': email_id,
                            'product ID': product_id,
                            'quantity': qty,
                            'status': status
                        })
                        order_lines.append((name, qty, status))
                except:
                    continue

        # Generate order confirmation email
        summary = "\n".join([f"- {n}: {q} ({s})" for n, q, s in order_lines])
        reply = f"Thank you for your order. Here's the status of your request:\n{summary}\n\nIf any item is out of stock, we will notify you when it becomes available."
        order_responses.append({"email ID": email_id, "response": reply})

# Task 3. Handle product inquiry

In [ ]:
import os
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

# Initialize the same embeddings object
embeddings = OpenAIEmbeddings(
    api_key='a0BIj000002bRQmMAM',
    base_url='https://47v4us7kyypinfb5lcligtc3x40ygqbs.lambda-url.us-east-1.on.aws/v1/'
)

# Create a wrapper function to make embeddings callable
class EmbeddingWrapper:
    def __init__(self, embeddings):
        self.embeddings = embeddings

    def __call__(self, text):
        return self.embeddings.embed_query(text)

# Use the wrapper
embedding_function = EmbeddingWrapper(embeddings)

# Path where the FAISS vector store is saved
VECTOR_STORE_PATH = "faiss_store"

# Load the FAISS vector store with the embedding function wrapper
# Note the parameter name is 'embeddings'
db = FAISS.load_local(VECTOR_STORE_PATH, embeddings=embedding_function)

# Search top 3 relevant products using vector similarity and generate GPT response
inquiry_responses = []

for _, row in emails_df.iterrows():
    email_id, subject, message = row['email_id'], row['subject'], row['message']
    category_row = next((item for item in email_classification if item['email ID'] == email_id), None)

    # Check if the category is 'product inquiry'
    if category_row and category_row['category'] == 'product inquiry':
        try:
            # Perform similarity search in the vector store
            docs = db.similarity_search(message, k=3)  # Top 3 most relevant products, since we have limited tokens!

            # Prepare context for the GPT prompt
            context = "\n".join([doc.page_content for doc in docs])

            # Generate GPT response using the provided context
            prompt = f"Customer question: {message}\n\nAnswer using this context:\n{context}"
            response = ask_gpt(prompt)  #  use ask_gpt for response

            # Append the response to the inquiry_responses list
            inquiry_responses.append({"email ID": email_id, "response": response})
        except Exception as e:
            print(f"Error processing email {email_id}: {e}")
            inquiry_responses.append({"email ID": email_id, "response": f"Error processing this inquiry: {str(e)}"})

### Writing Final Output Sheets

In [ ]:
# Create output Google Sheet with all result tabs
output_doc = gc.create('Solving Business Problems with AI - Output')

email_class_sheet = output_doc.add_worksheet("email-classification", rows=50, cols=2)
email_class_sheet.update([['email ID', 'category']], 'A1:B1')
set_with_dataframe(email_class_sheet, email_classification_df)

order_status_sheet = output_doc.add_worksheet("order-status", rows=50, cols=4)
order_status_sheet.update([['email ID', 'product ID', 'quantity', 'status']], 'A1:D1')
set_with_dataframe(order_status_sheet, pd.DataFrame(order_status))

order_response_sheet = output_doc.add_worksheet("order-response", rows=50, cols=2)
order_response_sheet.update([['email ID', 'response']], 'A1:B1')
set_with_dataframe(order_response_sheet, pd.DataFrame(order_responses))

inquiry_response_sheet = output_doc.add_worksheet("inquiry-response", rows=50, cols=2)
inquiry_response_sheet.update([['email ID', 'response']], 'A1:B1')
set_with_dataframe(inquiry_response_sheet, pd.DataFrame(inquiry_responses))

output_doc.share('', perm_type='anyone', role='reader')
print(f"\n All done! Shareable Output Link:\nhttps://docs.google.com/spreadsheets/d/{output_doc.id}")


 All done! Shareable Output Link:
https://docs.google.com/spreadsheets/d/1qGjp7CZea9ZE2LKXRPD1barz05I3eL5KDDKkPjA4o_I
